In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
import os
from PIL import Image
import numpy as np
import pandas as pd
from sklearn import datasets, model_selection

In [3]:
dirs = ['ants', 'bees']
data = []
label = []
import matplotlib.pyplot as plt
for i, d in enumerate(dirs):
    files = os.listdir('../data/hymenoptera_data/' + d)
    
    for f in files:
        img = Image.open('../data/hymenoptera_data/' + d + '/' + f, 'r')
        resize_img = img.resize((128, 128))
        r,g,b = resize_img.split()
        r_resize_img = np.asarray(np.float32(r) / 255.0)
        g_resize_img = np.asarray(np.float32(g) / 255.0)
        b_resize_img = np.asarray(np.float32(b) / 255.0)
        rgb_resize_img = np.asarray([r_resize_img, g_resize_img, b_resize_img])
        
        data.append(rgb_resize_img)
        label.append(i)

pd.DataFrame(data[0][0])

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.317647,0.321569,0.313726,0.317647,0.317647,0.333333,0.333333,0.321569,0.321569,0.321569,...,0.313726,0.305882,0.298039,0.305882,0.290196,0.313726,0.317647,0.305882,0.313726,0.313726
1,0.317647,0.317647,0.317647,0.317647,0.325490,0.321569,0.321569,0.321569,0.329412,0.317647,...,0.317647,0.309804,0.309804,0.305882,0.305882,0.309804,0.305882,0.313726,0.321569,0.301961
2,0.317647,0.317647,0.321569,0.313726,0.317647,0.317647,0.313726,0.325490,0.317647,0.321569,...,0.317647,0.301961,0.321569,0.305882,0.294118,0.298039,0.305882,0.301961,0.301961,0.305882
3,0.317647,0.309804,0.317647,0.317647,0.325490,0.329412,0.329412,0.333333,0.321569,0.321569,...,0.309804,0.313726,0.301961,0.309804,0.309804,0.317647,0.301961,0.301961,0.301961,0.305882
4,0.321569,0.325490,0.321569,0.321569,0.325490,0.333333,0.337255,0.325490,0.325490,0.329412,...,0.325490,0.317647,0.317647,0.298039,0.301961,0.294118,0.298039,0.313726,0.321569,0.305882
5,0.329412,0.325490,0.325490,0.329412,0.329412,0.329412,0.329412,0.321569,0.325490,0.317647,...,0.313726,0.313726,0.313726,0.305882,0.301961,0.301961,0.301961,0.305882,0.305882,0.301961
6,0.325490,0.329412,0.337255,0.333333,0.321569,0.329412,0.325490,0.317647,0.325490,0.345098,...,0.321569,0.313726,0.305882,0.309804,0.305882,0.317647,0.298039,0.309804,0.313726,0.301961
7,0.329412,0.325490,0.333333,0.325490,0.325490,0.329412,0.329412,0.337255,0.325490,0.333333,...,0.313726,0.321569,0.317647,0.313726,0.317647,0.309804,0.298039,0.301961,0.313726,0.305882
8,0.321569,0.337255,0.325490,0.321569,0.321569,0.337255,0.321569,0.333333,0.333333,0.329412,...,0.317647,0.317647,0.317647,0.317647,0.317647,0.309804,0.305882,0.301961,0.305882,0.298039
9,0.329412,0.333333,0.317647,0.321569,0.333333,0.325490,0.329412,0.333333,0.341176,0.329412,...,0.313726,0.317647,0.321569,0.313726,0.317647,0.321569,0.317647,0.309804,0.294118,0.305882


In [4]:
data = np.array(data, dtype='float32')
label = np.array(label, dtype='int64')

In [5]:
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(data, label, test_size=0.1)
print(len(train_X))
print(len(test_X))

357
40


In [6]:
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()
train_X.shape, train_Y.shape

(torch.Size([357, 3, 128, 128]), torch.Size([357]))

In [7]:
train = TensorDataset(train_X, train_Y)
train[0]

(tensor([[[0.0471, 0.0431, 0.0431,  ..., 0.8824, 0.8902, 0.8863],
          [0.0471, 0.0431, 0.0471,  ..., 0.8824, 0.8863, 0.8824],
          [0.0431, 0.0471, 0.0588,  ..., 0.8745, 0.8824, 0.8824],
          ...,
          [0.3176, 0.3137, 0.3176,  ..., 0.7608, 0.8235, 0.8510],
          [0.3255, 0.3137, 0.3137,  ..., 0.8078, 0.8471, 0.8235],
          [0.3333, 0.3176, 0.3176,  ..., 0.8314, 0.8510, 0.7451]],
 
         [[0.0510, 0.0471, 0.0471,  ..., 0.8745, 0.8784, 0.8824],
          [0.0510, 0.0471, 0.0510,  ..., 0.8745, 0.8745, 0.8784],
          [0.0471, 0.0588, 0.0706,  ..., 0.8667, 0.8745, 0.8706],
          ...,
          [0.3098, 0.3059, 0.3020,  ..., 0.7098, 0.7765, 0.8118],
          [0.3137, 0.3059, 0.3059,  ..., 0.7686, 0.8196, 0.7843],
          [0.3216, 0.3098, 0.3098,  ..., 0.8118, 0.8118, 0.7020]],
 
         [[0.0196, 0.0157, 0.0157,  ..., 0.7922, 0.8039, 0.8118],
          [0.0196, 0.0157, 0.0196,  ..., 0.7843, 0.8000, 0.8000],
          [0.0157, 0.0235, 0.0431,  ...,

In [8]:
train_loader = DataLoader(train, batch_size=32, shuffle=True)

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 5)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.fc1 = nn.Linear(20 * 29 * 29, 50)
        self.fc2 = nn.Linear(50, 2)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 20 * 29 * 29)
        x = F.relu(self.fc1(x))
        return F.log_softmax(self.fc2(x))

model = Net().cuda()

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
for epoch in range(200):
    total_loss = 0
    for train_x, train_y in train_loader:
        train_x, train_y = Variable(train_x.cuda()), Variable(train_y.cuda())
        optimizer.zero_grad()
        output = model(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    if (epoch+1) % 50 == 0:
        print(epoch+1, total_loss)

c:\users\jinsoo\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


50 0.013527185976272449
100 0.001995592088860576
150 0.0007850433594285278
200 0.00040180770793085685


In [12]:
test_X = np.array(test_X, dtype='float32')
test_Y = np.array(test_Y, dtype='int64')

test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

print(test_X.shape)
print(test_Y.shape)

torch.Size([40, 3, 128, 128])
torch.Size([40])


In [13]:
test_x, test_y = Variable(test_X.cuda()), Variable(test_Y.cuda())
result = torch.max(model(test_x).data, 1)[1]
accuracy = sum(test_y.cpu().data.numpy() == result.cpu().numpy()) / len(test_y.cpu().data.numpy())
accuracy

c:\users\jinsoo\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


0.625